In [ ]:
!pip install transformers
# !pip install tensorflow

In [ ]:
!ls /content/drive/MyDrive/app_reviews_all_annotated2.csv

/content/drive/MyDrive/app_reviews_all_annotated2.csv


In [ ]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall
from tensorflow.keras import metrics 
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Input, Dropout, Dense
import tensorflow.keras.activations as activations

In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

In [ ]:
# Import data
df = pd.read_csv('/content/drive/MyDrive/app_reviews_all_annotated2.csv')
df = df[['review', 'argument_cat', 'decision_cat']]

# Remove missing rows
df = df.dropna()

df = df.groupby('argument_cat').filter(lambda x : len(x) > 1)
df = df.groupby('decision_cat').filter(lambda x : len(x) > 1)

# Convert to numeric for bert
df['Argument'] = pd.Categorical(df['argument_cat'])
df['Decision'] = pd.Categorical(df['decision_cat'])
df['argument_cat'] = df['Argument'].cat.codes
df['decision_cat'] = df['Decision'].cat.codes

# Split into training and testing 
df, df_test = train_test_split(df, test_size = 0.1, stratify=df[['argument_cat']])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# BERT model
modelName = 'bert-base-uncased'
maxLen = 100
conf = BertConfig.from_pretrained(modelName)
conf.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = modelName, config = conf)

# Load transformer BERT model
transformerModel = TFBertModel.from_pretrained(modelName, config = conf)

bert = transformerModel.layers[0]
inputIds = Input(shape=(maxLen,), name='input_ids', dtype='int32')
inputs = {'input_ids': inputIds}

bertModel = bert(inputs)[1]
dropout = Dropout(conf.hidden_dropout_prob, name='pooled_output')
pooledOutput = dropout(bertModel, training=False)

arg = Dense(units=len(df.argument_cat.value_counts()), kernel_initializer=TruncatedNormal(stddev=conf.initializer_range), name='argument',activation='sigmoid')(pooledOutput)
dec = Dense(units=len(df.decision_cat.value_counts()), kernel_initializer=TruncatedNormal(stddev=conf.initializer_range), name='decision',activation='sigmoid')(pooledOutput)

#arg = activations.sigmoid(arg)
#dec = activations.sigmoid(dec)

outputs = {'argument': arg, 'decision': dec}

# Show model
model = Model(inputs=inputs, outputs=outputs, name='BERT_For_App_Review_Classification')
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "BERT_For_App_Review_Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
__________________________________________________________________________________________________
pooled_output (Dropout)         (None, 768)          0           bert[0][1]                       
__________________________________________________________________________________________________
argument (Dense)          

In [ ]:
# Training
optimizer = Adam(learning_rate=5e-05, epsilon=1e-08, decay=0.01, clipnorm=1.0)
loss = {'argument': CategoricalCrossentropy(from_logits = False), 'decision': CategoricalCrossentropy(from_logits = False)}
#metric = {'argument': CategoricalAccuracy('accuracy'), 'decision': CategoricalAccuracy('accuracy')}
metric = {'argument': tfa.metrics.F1Score(num_classes=4,
    average=None,
    threshold=0.5,
    name='f1_score')
, 'decision': tfa.metrics.F1Score(
    num_classes=5,
    average = None,
    threshold=0.5,
    name= 'f1_score'
)}
model.compile(optimizer = optimizer, loss = loss, metrics = metric)

x = tokenizer(text=df.review.to_list(), add_special_tokens=True, max_length=maxLen, 
              truncation=True, padding=True, return_tensors='tf', return_token_type_ids = False,
              return_attention_mask = True, verbose = True)

history = model.fit(x={'input_ids': x['input_ids']}, y={'argument': to_categorical(df.argument_cat), 'decision': to_categorical(df.decision_cat)},
                    validation_split=0.1, batch_size=64, epochs=40)

Epoch 1/40
35/35 [==============================] - ETA: 0s - loss: 1.9319 - argument_loss: 0.5513 - decision_loss: 1.3806 - argument_f1_score: 0.3015 - decision_f1_score: 0.2508

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


35/35 [==============================] - 64s 1s/step - loss: 1.9319 - argument_loss: 0.5513 - decision_loss: 1.3806 - argument_f1_score: 0.3015 - decision_f1_score: 0.2508 - val_loss: 1.7396 - val_argument_loss: 0.3353 - val_decision_loss: 1.4044 - val_argument_f1_score: 0.2991 - val_decision_f1_score: 0.2454
Epoch 2/40
35/35 [==============================] - 44s 1s/step - loss: 1.6577 - argument_loss: 0.3541 - decision_loss: 1.3036 - argument_f1_score: 0.3060 - decision_f1_score: 0.2905 - val_loss: 1.5216 - val_argument_loss: 0.3280 - val_decision_loss: 1.1936 - val_argument_f1_score: 0.3343 - val_decision_f1_score: 0.4381
Epoch 3/40
35/35 [==============================] - 44s 1s/step - loss: 1.2024 - argument_loss: 0.3181 - decision_loss: 0.8843 - argument_f1_score: 0.3204 - decision_f1_score: 0.4815 - val_loss: 0.8635 - val_argument_loss: 0.2885 - val_decision_loss: 0.5749 - val_argument_f1_score: 0.3302 - val_decision_f1_score: 0.6367
Epoch 4/40
35/35 [===========================

In [ ]:
# Evaluation
testArg = to_categorical(df_test['argument_cat'], 4)
testDec = to_categorical(df_test['decision_cat'])
testReview = tokenizer(text=df_test['review'].to_list(), add_special_tokens=True,
                         max_length=maxLen, truncation=True, padding=True, 
                         return_tensors='tf', return_token_type_ids = False,
                         return_attention_mask = False, verbose = True)
modelEval = model.evaluate(x={'input_ids': testReview['input_ids']},
                                 y={'argument': testArg, 'decision': testDec})

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/metrics.py:257: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


9/9 [==============================] - 2s 205ms/step - loss: 0.7552 - argument_loss: 0.3279 - decision_loss: 0.4274 - argument_f1_score: 0.4085 - decision_f1_score: 0.8393
